In [9]:
from cognite.neat import NeatSession, get_cognite_client

In [10]:
client = get_cognite_client(".env")

Found .env file in repository root. Loaded variables from .env file.


In [11]:
session = NeatSession(client)

session.read.cdf.data_model(("cdf_cdm", "CogniteCore", "v1"))

CDF <class 'tuple'> ('cdf_cdm', 'CogniteCore', 'v1') read successfully


[]

In [12]:
session.verify()

,identifier,resourceType,propertyName,referredBy,referredType,NeatIssue
0,"{'space': 'cdf_cdm', 'externalId': 'CogniteAss...",view,object3D,"{'source': {'space': 'cdf_cdm', 'externalId': ...",view property,PropertyNotFoundWarning
1,"{'space': 'cdf_cdm', 'externalId': 'CogniteCAD...",view,model3D,"{'source': {'space': 'cdf_cdm', 'externalId': ...",view property,PropertyNotFoundWarning
2,"{'space': 'cdf_cdm', 'externalId': 'CognitePoi...",view,model3D,"{'source': {'space': 'cdf_cdm', 'externalId': ...",view property,PropertyNotFoundWarning
3,"{'space': 'cdf_cdm', 'externalId': 'Cognite360...",view,model3D,"{'source': {'space': 'cdf_cdm', 'externalId': ...",view property,PropertyNotFoundWarning


In [13]:
session.prepare.data_model.to_extension(new_data_model_id=("sp_anders_space", "ExtensionCore", "v1"))

In [14]:
session.to.cdf.data_model()

C:\Users\AndersAlbert\Projects\internal\neat\cognite\neat\_rules\models\dms\_exporter.py:220: PropertyNotFoundWarning: (ViewId(space='cdf_cdm', external_id='CogniteAsset', version='v1'), 'view', 'object3D', PropertyId(source=ViewId(space='cdf_cdm', external_id='Cognite3DObject', version='v1'), property='asset'), 'view property')
  view_property = self._create_view_property(prop, view_properties_by_id)
C:\Users\AndersAlbert\Projects\internal\neat\cognite\neat\_rules\models\dms\_exporter.py:220: PropertyNotFoundWarning: (ViewId(space='cdf_cdm', external_id='CogniteCADRevision', version='v1'), 'view', 'model3D', PropertyId(source=ViewId(space='cdf_cdm', external_id='CogniteCADModel', version='v1'), property='revisions'), 'view property')
  view_property = self._create_view_property(prop, view_properties_by_id)
C:\Users\AndersAlbert\Projects\internal\neat\cognite\neat\_rules\models\dms\_exporter.py:220: PropertyNotFoundWarning: (ViewId(space='cdf_cdm', external_id='CognitePointCloudRevisio

,name,created,unchanged,failed_changed,error_messages
0,spaces,1.0,2.0,NaN,NaN
1,containers,NaN,2.0,27.0,1.0
2,views,NaN,NaN,33.0,1.0
3,data_models,1.0,NaN,NaN,NaN


In [15]:
from cognite.neat._utils.cdf.loaders import SpaceLoader

In [16]:
SpaceLoader(client).clean("sp_anders_space")

Deleted 1 data models
Deleted space ['sp_anders_space']


In [5]:

session.to.cdf.data_model()
# OR you can do
session.to.excel(filename)

AttributeError: 'CDFReadAPI' object has no attribute 'data_model'